6 comb

* vectorization: 
    a. bow
    b. tfidf
    * feature selection
        a. chi2
        b. f-test
        c. mi
        models:
            # logistic: 1008 comb
            # svm: 2688
            # rf: 8000
            
            
total: 82560 combs
lots of 5 cross validation = g


In [49]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
import re
from data_cleaning import *
import warnings

warnings.filterwarnings('ignore')

# 1. Preprocessing

In [50]:
# read data
train_data = pd.read_csv("Train.csv", sep=',')
test_data = pd.read_csv("Test.csv", sep=',')

In [51]:
#separating instance and label for Train
X_train_raw = [x[0] for x in train_data[['text']].values]
Y_train = [x[0] for x in train_data[['sentiment']].values]
X_test_raw = [x[0] for x in test_data[['text']].values]

#### (1). data cleaning

In [52]:
# 1. data cleaning (optional)
X_train_need_to_clean = pd.DataFrame(X_train_raw)
X_test_need_to_clean = pd.DataFrame(X_test_raw)

# remove url, # and @
X_train_need_to_clean.replace("\b*https?:\S*", ' ', regex=True, inplace=True)
X_train_need_to_clean.replace("\b*@\S*", ' ', regex=True, inplace=True)
# X_train_need_to_clean.replace("\b*#\S*", ' ', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*https?:\S*", ' ', regex=True, inplace=True)
X_test_need_to_clean.replace("\b*@\S*", ' ', regex=True, inplace=True)
# X_test_need_to_clean.replace("\b*#\S*", ' ', regex=True, inplace=True)


X_train_need_to_clean.replace('[^\w\s]','', regex=True, inplace=True)
X_test_need_to_clean.replace('[^\w\s]','', regex=True, inplace=True)
    
    
X_train_clean = [x[0] for x in X_train_need_to_clean[[0]].values]
X_test_clean = [x[0] for x in X_test_need_to_clean[[0]].values]

#### (2). vectorization (transformation)

In [53]:
#TFIDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2,2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_clean)
X_test_tfidf = tfidf_vectorizer.transform(X_test_clean)
X_train_tranformed, X_test_transformed = X_train_tfidf, X_test_tfidf


# 2. feature selection

### type of selector (choose one of them)

In [54]:
# chi2     
selector = SelectKBest(chi2,k=10000)
X_train_new = selector.fit_transform(X_train_tranformed,Y_train)
X_test_new = selector.transform(X_test_transformed)

# 3. training

### split data for validation

In [55]:
train_size = X_train_tranformed.shape[0]
test_size = X_test_transformed.shape[0]
## random hold out
ts = test_size/train_size
X_train_, X_validation, y_train_, y_validation  = train_test_split(X_train_new,Y_train, test_size=ts)

### Undersampling

In [157]:
from imblearn.under_sampling import RandomUnderSampler, NearMiss
sampling_strategy = {'negative':2662, 'positive':2500, 'neutral': 2000}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy,random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train_s, y_train_s)
X_train_s, y_train_s = X_resampled, y_resampled

In [158]:
resampled_labels = pd.DataFrame(y_train_s)
resampled_labels.value_counts()

negative    2662
positive    2500
neutral     2000
dtype: int64

### oversampling

In [88]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_, y_train_)

In [89]:
resampled_labels = pd.DataFrame(y_train_smote)
resampled_labels.value_counts()

negative    9144
neutral     9144
positive    9144
dtype: int64

In [9]:
X_train_s, y_train_s = X_train_, y_train_

In [10]:
cv = round(X_train_s.shape[0] / X_validation.shape[0])

## base model: 0R

In [56]:
clf = DummyClassifier(strategy='most_frequent')
basemodel = clf.fit(X_train_raw, Y_train)
print("base model score: ", basemodel.score(X_train_raw, Y_train))

base model score:  0.5806348041464086


In [57]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

## naive bayes

In [58]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB

# gussian
gnb = GaussianNB()
gnb.fit(X_train_s.toarray(), y_train_s)
gnb.score(X_validation.toarray(), y_validation)
print("Gaussian: ", gnb.score(X_validation.toarray(), y_validation))

# multinomial
mnb = MultinomialNB()
mnb.fit(X_train_s.toarray(), y_train_s)
mnb.score(X_validation.toarray(), y_validation)
print("multinomial: ", mnb.score(X_validation.toarray(), y_validation))

# complement
cnb = ComplementNB()
cnb.fit(X_train_s.toarray(), y_train_s)
cnb.score(X_validation.toarray(), y_validation)
print("complement: ", cnb.score(X_validation.toarray(), y_validation))

# bernoulli
bnb = BernoulliNB()
bnb.fit(X_train_s.toarray(), y_train_s)
print("bernoulli: ", bnb.score(X_validation.toarray(), y_validation))

# # categorical
# canb = CategoricalNB()
# canb.fit(X_train_s.toarray(), y_train_s)
# canb.score(X_validation.toarray(), y_validation)
# print("categorical: ", canb.score(X_validation.toarray(), y_validation))

Gaussian:  0.7696343662895556
multinomial:  0.5958353828496474
complement:  0.484833579275291
bernoulli:  0.6835546810952615


## decision tree

In [59]:
from sklearn.tree import DecisionTreeClassifier

dt_hyper = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2,20,2),
    'min_samples_leaf': range(1,10,1),
    'max_features': ['auto', 'log2']
}

# random method
search_dt = RandomizedSearchCV(DecisionTreeClassifier(), dt_hyper, scoring='f1_weighted', cv=cv, n_iter=800, n_jobs=-1)
dt_result = search_dt.fit(X_train_s, y_train_s)
dt_result.best_params_
dt_result.best_score_

0.45616441931138035

In [60]:
dt_result.best_score_

0.45616441931138035

In [61]:
dt_result.best_params_

{'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 16,
 'criterion': 'gini'}

In [62]:
dt_model = DecisionTreeClassifier(criterion= dt_result.best_params_['criterion'], max_depth=dt_result.best_params_['max_depth'],
                                  min_samples_leaf=dt_result.best_params_['min_samples_leaf'],
                                  max_features=dt_result.best_params_['max_features']).fit(X_train_s, y_train_s)

## logistic regression

In [63]:
from sklearn.linear_model import LogisticRegression

logi_hyper = {
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'penalty': ['l1', 'l2', 'none', 'elasticnet'],
    'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
    'max_iter': [100, 300, 500],
    'multi_class': ['auto', 'ovr', 'multinomial']
}

search_logi = RandomizedSearchCV(LogisticRegression(n_jobs=-1),logi_hyper, scoring='f1_weighted', cv=cv, n_iter=150, n_jobs=-1)
logi_result = search_logi.fit(X_train_s, y_train_s)

print(logi_result.best_params_)
print(logi_result.best_score_)

{'solver': 'lbfgs', 'penalty': 'none', 'multi_class': 'ovr', 'max_iter': 100, 'C': 0.01}
0.6212483408429171


In [64]:
pd.DataFrame(y_train_s).value_counts()

neutral     9112
positive    3947
negative    2644
dtype: int64

### SVM

In [65]:
from sklearn.svm import SVC

svm_hyper = {
    'degree': [3, 5, 10, 15],
    'gamma': [1,0.1,0.01,0.001],
    'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000],
    'kernel': ['linear', 'poly', 'rbf', 'sigmod'],
    'max_iter': [100, 500, 1000],
    'decision_function_shape': ['ovo', 'ovr']
}

# random method
search_svm = RandomizedSearchCV(SVC(), svm_hyper, scoring='f1_weighted', cv=cv, n_iter=200)
svm_result = search_svm.fit(X_train_s, y_train_s)

pd.DataFrame(svm_result.cv_results_)
print(svm_result.best_params_)
print(svm_result.best_score_)

{'max_iter': 1000, 'kernel': 'rbf', 'gamma': 1, 'degree': 3, 'decision_function_shape': 'ovr', 'C': 1000}
0.3879547141125343


### Random Forest

In [67]:
from sklearn.ensemble import RandomForestClassifier

rf_hyper = {
    'n_estimators': [90, 100, 115 , 130],
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2,20,2),
    'min_samples_leaf': range(1,10,1),
    'min_samples_split': range(2,10,2),
    'max_features': ['auto', 'log2']
}

# random method
search_rf = RandomizedSearchCV(RandomForestClassifier(n_jobs=-1), rf_hyper, scoring='f1_weighted', cv=cv, n_iter=800, n_jobs=-1)
rf_result = search_rf.fit(X_train_s, y_train_s)

pd.DataFrame(rf_result.cv_results_)
print(rf_result.best_params_)
print(rf_result.best_score_)

KeyboardInterrupt: 

### Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier


rf_model = RandomForestClassifier(criterion=rf_result.best_params_['criterion'], max_depth=rf_result.best_params_['max_depth'],
                                  max_features='log2', min_samples_leaf=rf_result.best_params_['min_samples_leaf'], 
                                  min_samples_split=rf_result.best_params_['min_samples_split'], n_estimators=rf_result.best_params_['n_estimators'],
                                  random_state=0, class_weight='balanced_subsample').fit(X_train_s, y_train_s)

# svm_model = SVC(degree=svm_result.best_params_['degree'], gamma=svm_result.best_params_['gamma'] ,C=svm_result.best_params_['C'], 
#                 kernel=svm_result.best_params_['kernel'], max_iter=svm_result.best_params_['max_iter'], 
#                 decision_function_shape=svm_result.best_params_['decision_function_shape']).fit(X_train_s, y_train_s)

svm_model = SVC( class_weight='balanced').fit(X_train_s, y_train_s)

logi_model = LogisticRegression(solver=logi_result.best_params_['solver'], penalty=logi_result.best_params_['penalty'],
                                C=logi_result.best_params_['C'], max_iter=logi_result.best_params_['max_iter'],
                                multi_class=logi_result.best_params_['multi_class'], ).fit(X_train_s, y_train_s)

estimators = [('rf', rf_model),('svr', svm_model), ('log', logi_model),('dt', dt_model)]

In [ ]:
dt_stacking = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier(), n_jobs=-1).fit(X_train_s, y_train_s)
dt_stacking.score(X_validation, y_validation)

In [ ]:
svm_stacking = StackingClassifier(estimators=estimators, final_estimator=SVC(), n_jobs=-1).fit(X_train_s, y_train_s)
svm_stacking.score(X_validation, y_validation)

In [ ]:
logi_stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=200), n_jobs=-1).fit(X_train_s, y_train_s)
logi_stacking.score(X_validation, y_validation)

super_estimator = [('ls', logi_stacking), ('ss', svm_stacking), ('ds', dt_stacking),('log', logi_model) ]
super_stacking = StackingClassifier(estimators=super_estimator, final_estimator=LogisticRegression(max_iter=200,n_jobs=-1)).fit(X_train_s, y_train_s)
super_stacking.score(X_validation, y_validation)                                    

# 4. Evaluation

### score

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import numpy as np
from mlxtend.evaluate import bias_variance_decomp

# this function will take the model and datas, print the evaluation scores and output the precdtion

def evaluate_model(model_name: str, model, X_validation, y_validation, X_test_new, cv=5):
    # basestic
    print(f"{model_name}: ")

    # validation acc
    validation_scores = cross_val_score(model,X_validation, y_validation, cv=cv)
    vc = np.mean(validation_scores)
    print("Validation accuracy: ", validation_scores)
    
    print("validation acc for each cross:", )

    # predictions for test data
    prediction = model.predict(X_test_new)

    validation_label = model.predict(X_validation)
    
    # individual metrics
    precision_m = precision_score(y_validation,validation_label, average = None)
    print('individual precision is ', precision_m)

    recall_m = recall_score(y_validation,validation_label, average = None)
    print('individual recall is ', recall_m)

    # f1
    f1_m = f1_score(y_validation,validation_label, average = None)
    print('individual f1 is ', f1_m)
    
    # macro metrics
    precision_m = precision_score(y_validation,validation_label, average = 'macro')
    print('macro precision is ', precision_m)

    recall_m = recall_score(y_validation,validation_label, average = 'macro')
    print('macro recall is ', recall_m)

    # f1
    f1_m = f1_score(y_validation,validation_label, average = 'macro')
    print('macro f1 is ', f1_m)


    # weighted metrics
    precision_w = precision_score(y_validation,validation_label, average = 'weighted')
    print('weighted precision is ', precision_w)

    recall_w = recall_score(y_validation,validation_label, average = 'weighted')
    print('weighted recall is ', recall_w)

    # f1
    f1_w = f1_score(y_validation,validation_label, average = 'weighted')
    print('weighted f1 is ', f1_w)

    
    
    return prediction

## basic models

In [ ]:
base_label = evaluate_model("basemodel 0R", basemodel, X_validation, y_validation,X_test_new)
dt_label = evaluate_model("decision tree", dt_model,X_validation, y_validation,X_test_new)
# print(evaluate_model("guassian nb", gnb, X_validation.toarray(), y_validation_numberise, X_test_new.toarray()))

In [ ]:
logi_label = evaluate_model("Logistic Regression", logi_model, X_validation, y_validation,X_test_new)
s_label = evaluate_model("SVM", svm_model, X_validation, y_validation,X_test_new)

### bagging models

In [ ]:
rf_label = evaluate_model("random forest", rf_model, X_validation, y_validation,X_test_new)

### stacking

In [ ]:
ss_label = evaluate_model("svm_stacking", svm_stacking, X_validation, y_validation,X_test_new)
dts_label = evaluate_model("dt_stacking", dt_stacking, X_validation, y_validation,X_test_new)
logi_s_label = evaluate_model("logi_stacking", logi_stacking, X_validation, y_validation,X_test_new)

evaluate_model("super stacking", super_stacking, X_validation, y_validation,X_test_new)

plot_confusion_matrix(super_stacking, X_validation, y_validation)
plt.show()

# 5. Error Analysis

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = gnb.predict(X_validation.toarray())
print(y_pred)
# cm = confusion_matrix()
# plot_confusion_matrix(y_validation, y_pred)


In [ ]:
pd.DataFrame(y_pred).value_counts()

In [ ]:
# logistic regression
plot_confusion_matrix(logi_model, X_validation, y_validation)
plt.show()

In [ ]:
# svm
plot_confusion_matrix(svm_model, X_validation, y_validation)
plt.show()

In [ ]:
# random forest
plot_confusion_matrix(rf_model, X_validation, y_validation)
plt.show()

In [ ]:
# svm_stacking
plot_confusion_matrix(svm_stacking, X_validation, y_validation)
plt.show()


In [ ]:
# dt_stacking
plot_confusion_matrix(dt_stacking, X_validation, y_validation)
plt.show()

In [ ]:
# logi_stacking
plot_confusion_matrix(logi_stacking, X_validation, y_validation)
plt.show()

In [ ]:
pd.DataFrame(logi_label).value_counts()

## output

In [ ]:
import csv


prediction_label = dts_label
with open('prediction.csv','w') as output:
    output.write("id,sentiment\n")
    for i in range(0,len(prediction_label)):
        output.write(str(test_data['id'].iloc[i]))
        output.write(",")
        output.write(prediction_label[i])
        output.write('\n')

# k fold

In [ ]:
# from sklearn.model_selection import cross_validate
# _scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']


# logi_model = LogisticRegression(solver=logi_result.best_params_['solver'], penalty=logi_result.best_params_['penalty'],
#                                 C=logi_result.best_params_['C'], max_iter=logi_result.best_params_['max_iter'],
#                                 multi_class=logi_result.best_params_['multi_class'], class_weight = 'balanced')

# svm_model = SVC(degree=svm_result.best_params_['degree'], gamma=svm_result.best_params_['gamma'] ,C=svm_result.best_params_['C'], 
#                 kernel=svm_result.best_params_['kernel'], max_iter=svm_result.best_params_['max_iter'], 
#                 decision_function_shape=svm_result.best_params_['decision_function_shape'])

# rf_model = RandomForestClassifier(criterion=rf_result.best_params_['criterion'], max_depth=rf_result.best_params_['max_depth'],
#                                   max_features='log2', min_samples_leaf=rf_result.best_params_['min_samples_leaf'], 
#                                   min_samples_split=rf_result.best_params_['min_samples_split'], n_estimators=rf_result.best_params_['n_estimators'],
#                                   random_state=0)

# # logi_f_result = cross_validate(estimator=logi_model, X=X_train_new, y = Y_train, scoring=_scoring, return_train_score=True,cv=5, error_score="raise")

# estimators = [('rf', rf_model),('svr', svm_model), ('log', logi_model)]

# svm_stacking = StackingClassifier(estimators=estimators, final_estimator=SVC(), n_jobs=-1)
# svm_stacking_f_result = cross_validate(estimator=svm_stacking,X=X_train_new, y = Y_train, scoring=_scoring, return_train_score=True,cv=5, error_score="raise", n_jobs =-1)
